In [125]:
import glob
import psycopg2
import pandas as pd
import time

In [140]:
# Label Sample
file_path = 'D:\Final_Project\Data\Test_label\\test_label.txt'
with open(file_path, 'r') as f:
    data = f.readlines()
print(data)

['16 0.233333 0.708 0.253333 0.346667\n', '8 0.233333 0.708 0.253333 0.346667\n', '7 0.233333 0.708 0.253333 0.346667\n']


In [141]:
label_list = []
for a in data:
    label_list.append(int(a.split(' ')[0]))
label_list

[16, 8, 7]

In [142]:
# postgre DB 연결
try:
    conn = psycopg2.connect(host="localhost",database="machine129", user="postgres", password="admin",port=5432)
    print('===접속 성공===')
    cur = conn.cursor()
except psycopg2.DatabaseError as db_err:
    print('접속 오류 !!')
    print(db_err)

===접속 성공===


In [62]:
# Food table 컬럼명 조회 query
cur=conn.cursor()
try:
    cur.execute("Select COLUMN_NAME From INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'food' order by ordinal_position;")
    columns = cur.fetchall()
except:
    print('Error!')
print(columns)

[('class_num',), ('food_name',), ('serving_size_g',), ('calorie_kj',), ('calorie_kcal',), ('carbohydrate_g',), ('sugar_g',), ('protein_g',), ('fat_g',), ('saturated_fat_g',), ('polyunsaturated_fat_g',), ('unsaturated_fat_g',), ('cholesterol_mg',), ('dietary_fiber_g',), ('salt_mg',), ('potassium_mg',)]


In [63]:
# 음식 이름 찾기
cur=conn.cursor()
try:
    cur.execute(f"Select food_name From food WHERE class_num={food_class};")
    food_name = cur.fetchall()
except:
    print('Error!')

# 확인
print(food_name)

[('쌀밥',)]


In [64]:
# 영양소 확인
cur=conn.cursor()
try:
    cur.execute(f"Select * From food WHERE class_num={food_class};")
    ssalbab = cur.fetchall()
except:
    print('Error!')

# 확인
print(ssalbab)

[(16, '쌀밥', 210.0, 1130.0, 270.0, 61.0, 0.0, 5.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.0, 20.0, 0.0)]


In [97]:
# 쿼리 작성할 예정

# 문자열 제외한 컬럼들
cur=conn.cursor()
try:
    cur.execute("Select COLUMN_NAME From INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'Food' and data_type = 'double precision' order by ordinal_position;")
    fcolumns = cur.fetchall()
except:
    print('Error!')

# SELECT "요기" FROM ~
label_list = list(set(label_list)) # 라벨리스트 > 집합 > 라벨리스트 를 통해 중복값을 제거 해줌.
columns_query = ''
for i in fcolumns:
    columns_query += 'sum(' + i[0] + '), '
columns_query = columns_query.rstrip(', ')

# WHERE " 요기~ "
where_query = ''
for i in label_list:
    where_query += 'class_num = ' + str(i) + " or "
where_query = where_query.rstrip(' or ')

query = 'SELECT ' + columns_query + ' FROM food WHERE ' + where_query

In [143]:
query

'SELECT sum(serving_size_g), sum(calorie_kj), sum(calorie_kcal), sum(carbohydrate_g), sum(sugar_g), sum(protein_g), sum(fat_g), sum(saturated_fat_g), sum(polyunsaturated_fat_g), sum(unsaturated_fat_g), sum(cholesterol_mg), sum(dietary_fiber_g), sum(salt_mg), sum(potassium_mg) FROM food WHERE class_num = 16 or class_num = 8 or class_num = 7'

In [102]:
cur=conn.cursor()
try:
    cur.execute(query)
    score = cur.fetchall()
except:
    print('Error!')
print(score)

[(520.0, 1690.0, 404.0, 72.09, 2.05, 15.47, 7.14, 1.338, 2.1959999999999997, 2.227, 14.0, 1.0, 1296.0, 398.0)]


In [108]:
score[0]

(520.0,
 1690.0,
 404.0,
 72.09,
 2.05,
 15.47,
 7.14,
 1.338,
 2.1959999999999997,
 2.227,
 14.0,
 1.0,
 1296.0,
 398.0)

In [145]:
testid = 'testemail@gmail.com'
now = time.strftime('%Y-%m-%d %H:%M:%S')

valuestxt = "('" + testid + "', '" + now + "'"
for i in score[0]:
    valuestxt += ", "
    valuestxt += str(i)
valuestxt += ');'

insertquery = 'INSERT INTO meals VALUES ' + valuestxt

cur=conn.cursor()
try:
    cur.execute(insertquery)
    conn.commit()
except:
    print('Error!')
